In [10]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris() #데이터를 로드

features = iris.data # 특성과 타깃을 만듭니다
target = iris.target

thresholder = VarianceThreshold(threshold=.5) # 기준값을 만듭니다

features_high_variance = thresholder.fit_transform(features) # 기준값보다 높은 특성을 선택합니다.

print(features_high_variance[0:3]) # 선택한 특성을 확인
print(thresholder.variances_) # 분산 확인

[[5.1 1.4 0.2]
 [4.9 1.4 0.2]
 [4.7 1.3 0.2]]
[0.68112222 0.18871289 3.09550267 0.57713289]


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # 특성 행렬을 표준화합니다.
features_std = scaler.fit_transform(features)

selector = VarianceThreshold() # 각 특성의 분산을 계산합니다.
selector.fit(features_std).variances_
# 표준화하면 분산은 열단위로 1이 된다

array([1., 1., 1., 1.])

In [14]:
from sklearn.feature_selection import VarianceThreshold

features = [[0, 1, 0], # 예제 특성 행렬
		[0, 1, 1], # 특성 0: 80%가 클래스 0
		[0, 1, 0], # 특성 1: 80%가 클래스 1
		[0, 1, 1], # 특성 2: 60%가 클래스 0, 40%는 클래스 1
		[1, 0, 0]]

# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
print(thresholder.fit_transform(features))
print(thresholder.variances_)

import numpy as np
np.var(features, axis=0) #넘파이 var 함수를 사용하여 분산을 계산합니다

[[0]
 [1]
 [0]
 [1]
 [0]]
[0.16 0.16 0.24]


array([0.16, 0.16, 0.24])

In [15]:
import pandas as pd
import numpy as np

# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1], [2, 2, 0], [3, 3, 1], [4, 4, 0], [5, 5, 1],
		[6, 6, 0], [7, 7, 1], [8, 7, 0], [9, 7, 1]])

dataframe = pd.DataFrame(features) # 특성 행렬을 DataFrame으로 변환
corr_matrix = dataframe.corr().abs() # 상관관계 행렬을 만듭니다.
# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택합니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [23]:
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3) # 특성을 삭제합니다.

print(dataframe.corr()) #상관관계 행렬
print(upper) #상관관계 행렬의 상삼각 행렬

          0         1         2
0  1.000000  0.976103  0.000000
1  0.976103  1.000000 -0.034503
2  0.000000 -0.034503  1.000000
    0         1         2
0 NaN  0.976103  0.000000
1 NaN       NaN  0.034503
2 NaN       NaN       NaN


In [24]:
#상관관계 행렬은 넘파이 corrcoef()로 구할 수 있습니다.
#corrcoef()는 특성이 행에 놓여 있을 것으로 가정합니다.
#특성이 열에 놓여 있다고 알려주려면 rowvar 매개변수를 False로 지정합니다.
print(np.corrcoef(features, rowvar=False))

# np.triu()는 주어진 배열에서 상삼각 행렬을 추출하여 반환합니다.
# 매개변수 k가 기본값 0이면 반환되는 행렬에 대각원소가 포함됩니다.
# k값이 커질수록 대각원소에서 k만큼 떨어진 삼각행렬을 반환합니다.
# 예) k=2일 경우 주대각선에서 2만큼 떨어진 원소부터 포함됩니다.
print(np.triu(np.ones((4, 4)), k=2))

# np.tril()는 주어진 배열에서 하삼각 행렬을 추출 반환합니다.
print(np.tril(np.ones((4, 4)), k=0))

[[ 1.          0.97610336  0.        ]
 [ 0.97610336  1.         -0.03450328]
 [ 0.         -0.03450328  1.        ]]
[[0. 0. 1. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1. 0. 0. 0.]
 [1. 1. 0. 0.]
 [1. 1. 1. 0.]
 [1. 1. 1. 1.]]


In [25]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = load_iris() # 데이터 로드
features = iris.data
target = iris.target
features = features.astype(int) # 범주형 데이터를 정수형으로 변환

chi2_selector = SelectKBest(chi2, k=2) # 카이제곱 통계값이 가장 큰 특성 두 개를 선택
features_kbest = chi2_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [27]:
# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [28]:
# 특정 특성 개수를 선택하는 대신 Selectpercentile를 사용하여 특성의 상위 n 퍼센트를 선택할 수 있습니다.
from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)

print("원본 특성 개수:", features.shape[1]) # 결과 선택
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 3


In [31]:
target
#특성 행렬의 차원을 (3, 50, 4)로 바꾸어 클래스별 합을 구합니다.
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed

array([[230, 152,  50,   0],
       [274, 116, 191,  50],
       [304, 129, 255,  79]])

In [36]:
#특성 타깃과 전혀 관계없다면 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 됩니다.
expected = features.sum(axis=0) / 3
expected

array([269.33333333, 132.33333333, 165.33333333,  43.        ])

In [37]:
#카이제곱 공식에 위헤서 구한 observed와 expected를 대입합니다.
np.sum((observed - expected)**2 / expected, axis=0)

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [38]:
#카이제곱 값이 큰 세 번째, 네 번째 특성이 선택됩니다. chi2_selector객체의 scores_속성에 저장
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [42]:
#ANOVA 직접 계산
##전체 평균과 클래스 평균을 계산
total_mean = np.mean(features, axis=0)
print(total_mean)
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
print(class_mean)

[5.38666667 2.64666667 3.30666667 0.86      ]
[[4.6  3.04 1.   0.  ]
 [5.48 2.32 3.82 1.  ]
 [6.08 2.58 5.1  1.58]]


In [43]:
#ss_total 계산
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
print(ss_between)
ss_total = np.sum((features - total_mean)**2, axis=0)
print(ss_total)

[ 55.41333333  13.29333333 440.01333333  63.88      ]
[105.57333333  42.27333333 467.89333333  76.06      ]


In [41]:
#ss_beteen과 ss_tatal을 F-값 공식에 대입
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f

array([  81.19776715,   33.71497585, 1160.00645624,  385.48275862])

In [44]:
fvalue_selector.scores_ #F-값 scores_속성에서 확인

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

In [45]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model

# 특성 행렬과 타깃 벡터를 생성합니다.
# 선형회귀분석에 필요한 모의 데이터 생성
features, target = make_regression(n_samples = 10000,  
                                n_features = 100,
                                n_informative = 2,
                                random_state = 1)

# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression() 

# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")  # neg_mean_squared_error : 평균제곱오차
print(rfecv.fit(features, target))
print(rfecv.transform(features))

RFECV(cv=None,
      estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                 normalize=False),
      min_features_to_select=1, n_jobs=None, scoring='neg_mean_squared_error',
      step=1, verbose=0)
[[ 0.00850799  0.7031277  -0.925066  ]
 [-1.07500204  2.56148527  0.4746258 ]
 [ 1.37940721 -1.77039484 -0.39616889]
 ...
 [-0.80331656 -1.60648007  0.25068305]
 [ 0.39508844 -1.34564911 -1.35054293]
 [-0.55383035  0.82880112  0.14050409]]


In [46]:
print(rfecv.n_features_) # 최선의 특성 개수
print(rfecv.support_) # 선택된 특성이 표시된 불리언 마스크
print(rfecv.ranking_) # 특성의 순위: 최고(1)에서 최악(96)까지

3
[False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False]
[69 15 94 45 13  1 35  6 31 81 36 25  2  7 53 40 27 46 75 65 61 68 92  9
 39 48 98 96 47 71 19 37 11 20 50  4 33 42 67  1 43 63 85 86 56 60  5 16
  8 55 93 73 10 76  1 77 52 24 58 62 21 82 72 90 80 91 18 30 57 89 64 51
 59 17 28 32 49 66 87 84 38 88 34 44 14 79 41 12 29 23  3 78 22 95 26 70
 54 83 74 97]


In [47]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=3)
print(rfe.fit(features, target))
print(rfe.transform(features))
# rfe객체가 선택한 특성이 rfecv 객체가 선택한 특성과 동일한지 확인하기 위해 불리언 마스크를 비교
print(np.all(rfe.support_ == rfecv.support_))

RFE(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                               normalize=False),
    n_features_to_select=3, step=1, verbose=0)
[[ 0.00850799  0.7031277  -0.925066  ]
 [-1.07500204  2.56148527  0.4746258 ]
 [ 1.37940721 -1.77039484 -0.39616889]
 ...
 [-0.80331656 -1.60648007  0.25068305]
 [ 0.39508844 -1.34564911 -1.35054293]
 [-0.55383035  0.82880112  0.14050409]]
True
